# Análisis del Dataset Zoo - Clasificación Multiclase

Este notebook implementa y evalúa 6 algoritmos de clasificación en el dataset Zoo de UCI:
1. Naive Bayes Gaussiano
2. MLE Multivariante (Full Bayesian Gaussian)
3. Histogram Bayes
4. Parzen Windows
5. k-NN Density Bayes
6. k-NN Rule

Dataset: 17 atributos (15 binarios + 1 numérico + 1 clase), 7 clases de animales

## 1. Importación de librerías

In [90]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KernelDensity
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from scipy.stats import multivariate_normal
from sklearn.model_selection import cross_val_score

# Silenciar warnings
os.environ['LOKY_MAX_CPU_COUNT'] = '4'
warnings.filterwarnings('ignore', category=UserWarning, module='joblib')

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga y análisis exploratorio del dataset

In [91]:
# Clases del Zoo dataset (1-7 -> labels para report)
class_names = ['mammal', 'bird', 'reptile', 'fish', 'amphibian', 'invertebrate', 'insect']

print("=" * 80)
print("ANÁLISIS DEL DATASET ZOO (Multiclass Classification)")
print("=" * 80)

# Cargar datos: zoo.data (col 0: animal name (ignorar), col 1-17: features, col 18: class 1-7)
df = pd.read_csv('./data_zoo/zoo.csv')
# df = pd.read_csv('./data_zoo/zoo.csv')
df.columns = ['animal'] + [f'feature_{i}' for i in range(1, 17)] + ['class']
X = df.iloc[:, 1:-1]  # Features 1-17
y = df.iloc[:, -1].values - 1  # Clase 0-6 para sklearn

print("\nInformación del dataset:")
print(f"Forma: {X.shape} (instancias x features)")
print(f"Clases: {len(np.unique(y))} (multiclass: {class_names})")
print("\nDistribución de clases:")
unique, counts = np.unique(y, return_counts=True)
for i, (cls, count) in enumerate(zip(class_names, counts)):
    print(f"Clase {i+1} ({cls}): {count} muestras ({count/len(y)*100:.1f}%)")

# Mostrar primeras filas
print("\nPrimeras 5 filas del dataset:")
print(df.head())

ANÁLISIS DEL DATASET ZOO (Multiclass Classification)

Información del dataset:
Forma: (101, 16) (instancias x features)
Clases: 7 (multiclass: ['mammal', 'bird', 'reptile', 'fish', 'amphibian', 'invertebrate', 'insect'])

Distribución de clases:
Clase 1 (mammal): 41 muestras (40.6%)
Clase 2 (bird): 20 muestras (19.8%)
Clase 3 (reptile): 5 muestras (5.0%)
Clase 4 (fish): 13 muestras (12.9%)
Clase 5 (amphibian): 4 muestras (4.0%)
Clase 6 (invertebrate): 8 muestras (7.9%)
Clase 7 (insect): 10 muestras (9.9%)

Primeras 5 filas del dataset:
     animal  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0  aardvark          1          0          0          1          0          0   
1  antelope          1          0          0          1          0          0   
2      bass          0          0          1          0          0          1   
3      bear          1          0          0          1          0          0   
4      boar          1          0          0         

## 3. Configuración de validación cruzada

**Estrategia:** Usamos validación cruzada 5-fold estratificada sobre **todo el conjunto de datos**. En cada fold:
- Se divide en Train+Val (80%) y Test (20%)
- El conjunto Train+Val se subdivide para selección de hiperparámetros (validación interna)
- El Test de cada fold se usa para evaluación final

In [92]:
# Configuración: CV estratificado 5-fold sobre TODO el dataset
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)

print(f"Total de muestras: {len(X)}")
print(f"Estrategia: Cross-Validation 5-fold sobre todo el conjunto")
print(f"Cada fold: ~{len(X)*0.8:.0f} train+val (~80%), ~{len(X)*0.2:.0f} test (~20%)")
print("\nDistribución de clases en el conjunto completo:")
print(pd.Series(y).value_counts().sort_index())

print("\n✓ Configuración de validación cruzada: 5-fold estratificada sobre todo el dataset")

Total de muestras: 101
Estrategia: Cross-Validation 5-fold sobre todo el conjunto
Cada fold: ~81 train+val (~80%), ~20 test (~20%)

Distribución de clases en el conjunto completo:
0    41
1    20
2     5
3    13
4     4
5     8
6    10
Name: count, dtype: int64

✓ Configuración de validación cruzada: 5-fold estratificada sobre todo el dataset


## 4. Funciones auxiliares para evaluación con CV

In [93]:
# Función para evaluar modelo en un fold específico
def evaluate_fold(model, X_test, y_test, fold_num):
    """Evalúa el modelo en el conjunto de test del fold"""
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    f1_mac = f1_score(y_test, preds, average='macro')
    cm = confusion_matrix(y_test, preds)
    return acc, f1_mac, cm, preds

# Función para imprimir resultados agregados de CV
def print_cv_results(model_name, accuracies, f1_scores, all_preds, all_true):
    """Imprime resultados agregados de cross-validation"""
    print(f"\n{'='*80}")
    print(f"RESULTADOS CV: {model_name}")
    print(f"{'='*80}")
    print(f"\nAccuracy por fold: {[f'{acc:.4f}' for acc in accuracies]}")
    print(f"Accuracy promedio: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"\nF1-macro por fold: {[f'{f1:.4f}' for f1 in f1_scores]}")
    print(f"F1-macro promedio: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    
    # Reporte global (concatenando todas las predicciones)
    print(f"\n--- Reporte de clasificación global (todos los folds) ---")
    print(classification_report(all_true, all_preds, target_names=class_names))
    
    # Matriz de confusión global
    cm_global = confusion_matrix(all_true, all_preds)
    print(f"\nMatriz de confusión global:")
    print(cm_global)
    
    return np.mean(accuracies), np.mean(f1_scores)

print("✓ Funciones de evaluación definidas")

✓ Funciones de evaluación definidas


## 5. Modelo 1: Naive Bayes Gaussiano

In [94]:
print("=" * 80)
print("1. NAIVE BAYES GAUSSIANO")
print("=" * 80)

nb_accuracies = []
nb_f1_scores = []
nb_all_preds = []
nb_all_true = []

for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Entrenar Naive Bayes (sin hiperparámetros)
    nb = GaussianNB()
    nb.fit(X_train_val, y_train_val)
    
    # Evaluar en test del fold
    acc, f1, cm, preds = evaluate_fold(nb, X_test_fold, y_test_fold, fold)
    nb_accuracies.append(acc)
    nb_f1_scores.append(f1)
    nb_all_preds.extend(preds)
    nb_all_true.extend(y_test_fold)
    
    print(f"Accuracy: {acc:.4f}, F1-macro: {f1:.4f}")

# Resultados agregados
nb_mean_acc, nb_mean_f1 = print_cv_results("Naive Bayes", nb_accuracies, nb_f1_scores, 
                                            nb_all_preds, nb_all_true)

1. NAIVE BAYES GAUSSIANO

--- Fold 1/5 ---
Accuracy: 1.0000, F1-macro: 1.0000

--- Fold 2/5 ---
Accuracy: 1.0000, F1-macro: 1.0000

--- Fold 3/5 ---
Accuracy: 1.0000, F1-macro: 1.0000

--- Fold 4/5 ---
Accuracy: 0.9500, F1-macro: 0.8367

--- Fold 5/5 ---
Accuracy: 0.8500, F1-macro: 0.6583

RESULTADOS CV: Naive Bayes

Accuracy por fold: ['1.0000', '1.0000', '1.0000', '0.9500', '0.8500']
Accuracy promedio: 0.9600 ± 0.0583

F1-macro por fold: ['1.0000', '1.0000', '1.0000', '0.8367', '0.6583']
F1-macro promedio: 0.8990 ± 0.1360

--- Reporte de clasificación global (todos los folds) ---
              precision    recall  f1-score   support

      mammal       0.98      1.00      0.99        41
        bird       1.00      1.00      1.00        20
     reptile       0.60      0.60      0.60         5
        fish       0.93      1.00      0.96        13
   amphibian       1.00      0.75      0.86         4
invertebrate       1.00      1.00      1.00         8
      insect       1.00      0.9

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


## 6. Modelo 2: MLE Multivariante (Full Bayesian Gaussian)

In [95]:
print("=" * 80)
print("2. MLE MULTIVARIANTE (Full Bayesian Gaussian)")
print("=" * 80)

class FullGaussianBayes:
    def __init__(self):
        self.priors = None
        self.means = None
        self.covs = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.means = np.array([X[y == c].mean(axis=0) for c in self.classes])
        self.covs = np.array([np.cov(X[y == c].T) + 1e-6 * np.eye(X.shape[1]) for c in self.classes])
        return self
    
    def predict(self, X):
        n_samples = X.shape[0]
        ll = np.zeros((n_samples, len(self.classes)))
        for i, c in enumerate(self.classes):
            ll[:, i] = multivariate_normal(mean=self.means[i], cov=self.covs[i]).logpdf(X)
        posteriors = np.exp(ll) * self.priors
        posteriors /= posteriors.sum(axis=1, keepdims=True)
        return np.argmax(posteriors, axis=1)

print("✓ Clase FullGaussianBayes definida")

2. MLE MULTIVARIANTE (Full Bayesian Gaussian)
✓ Clase FullGaussianBayes definida


In [96]:
mle_accuracies = []
mle_f1_scores = []
mle_all_preds = []
mle_all_true = []

for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Entrenar MLE Full
    mle = FullGaussianBayes()
    mle.fit(X_train_val.values, y_train_val)
    
    # Evaluar en test del fold
    acc, f1, cm, preds = evaluate_fold(mle, X_test_fold.values, y_test_fold, fold)
    mle_accuracies.append(acc)
    mle_f1_scores.append(f1)
    mle_all_preds.extend(preds)
    mle_all_true.extend(y_test_fold)
    
    print(f"Accuracy: {acc:.4f}, F1-macro: {f1:.4f}")

# Resultados agregados
mle_mean_acc, mle_mean_f1 = print_cv_results("MLE Full Gaussian", mle_accuracies, mle_f1_scores,
                                              mle_all_preds, mle_all_true)


--- Fold 1/5 ---
Accuracy: 0.8571, F1-macro: 0.6463

--- Fold 2/5 ---
Accuracy: 0.8000, F1-macro: 0.5778

--- Fold 3/5 ---
Accuracy: 0.8000, F1-macro: 0.5778

--- Fold 3/5 ---
Accuracy: 0.8500, F1-macro: 0.6441

--- Fold 4/5 ---
Accuracy: 0.8000, F1-macro: 0.5143

--- Fold 5/5 ---
Accuracy: 0.8500, F1-macro: 0.6441

--- Fold 4/5 ---
Accuracy: 0.8000, F1-macro: 0.5143

--- Fold 5/5 ---


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/var/folders/0m/q5gmrh695pvc0bcq9s0zmcqh0000gr/T/ipykernel_2848/2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
/var/folders/0m/q5gmrh695pvc0bcq9s0zmcqh0000gr/T/ipykernel_2848/2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
/var/folders/0m/q5gmrh695pvc0bcq9s0zmcqh0000gr/T/ipykernel_2848/2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
/var/folders/0m/q5gmrh695pvc0bcq9s0zmcqh0000gr/T/ipykernel_2848/2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
/var/folders/0m/q

Accuracy: 0.7500, F1-macro: 0.5374

RESULTADOS CV: MLE Full Gaussian

Accuracy por fold: ['0.8571', '0.8000', '0.8500', '0.8000', '0.7500']
Accuracy promedio: 0.8114 ± 0.0390

F1-macro por fold: ['0.6463', '0.5778', '0.6441', '0.5143', '0.5374']
F1-macro promedio: 0.5840 ± 0.0540

--- Reporte de clasificación global (todos los folds) ---
              precision    recall  f1-score   support

      mammal       0.68      1.00      0.81        41
        bird       1.00      1.00      1.00        20
     reptile       0.00      0.00      0.00         5
        fish       1.00      0.85      0.92        13
   amphibian       0.00      0.00      0.00         4
invertebrate       1.00      0.50      0.67         8
      insect       1.00      0.60      0.75        10

    accuracy                           0.81       101
   macro avg       0.67      0.56      0.59       101
weighted avg       0.78      0.81      0.77       101


Matriz de confusión global:
[[41  0  0  0  0  0  0]
 [ 0 20  0

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## 7. Modelo 3: Histogram Bayes

In [97]:
print("=" * 80)
print("3. DENSIDAD NO PARAMÉTRICA - HISTOGRAMA")
print("=" * 80)

class HistogramBayes:
    def __init__(self, bins=2):
        self.bins = bins
        self.priors = None
        self.hist_per_class = None
        self.edges = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.hist_per_class = {}
        for c in self.classes:
            X_c = X[y == c]
            hists = []
            edges_list = []
            for feat in range(X.shape[1]):
                hist, edges = np.histogram(X_c.iloc[:, feat], bins=self.bins, density=True)
                hists.append(hist)
                edges_list.append(edges)
            self.hist_per_class[c] = (np.array(hists), edges_list)
        self.edges = edges_list[0] if edges_list else None
        return self
    
    def _density_hist(self, x, c):
        hists, edges = self.hist_per_class[c]
        dens = 1.0
        for i, feat_val in enumerate(x):
            bin_idx = np.digitize(feat_val, edges[i]) - 1
            if 0 <= bin_idx < len(hists[i]):
                dens *= hists[i][bin_idx]
            else:
                dens *= 0
        return dens
    
    def predict(self, X):
        n_samples = len(X)
        preds = np.zeros(n_samples, dtype=int)
        for i in range(n_samples):
            posteriors = []
            for c in self.classes:
                dens = self._density_hist(X.iloc[i], c)
                post = self.priors[c] * dens
                posteriors.append(post)
            preds[i] = self.classes[np.argmax(posteriors)]
        return preds

print("✓ Clase HistogramBayes definida")

3. DENSIDAD NO PARAMÉTRICA - HISTOGRAMA
✓ Clase HistogramBayes definida


In [98]:
hist_accuracies = []
hist_f1_scores = []
hist_all_preds = []
hist_all_true = []

for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Entrenar Histogram Bayes (bins=2 fijo para datos binarios)
    hist_bayes = HistogramBayes(bins=2)
    hist_bayes.fit(pd.DataFrame(X_train_val), y_train_val)
    
    # Evaluar en test del fold
    acc, f1, cm, preds = evaluate_fold(hist_bayes, pd.DataFrame(X_test_fold), y_test_fold, fold)
    hist_accuracies.append(acc)
    hist_f1_scores.append(f1)
    hist_all_preds.extend(preds)
    hist_all_true.extend(y_test_fold)
    
    print(f"Accuracy: {acc:.4f}, F1-macro: {f1:.4f}")

# Resultados agregados
hist_mean_acc, hist_mean_f1 = print_cv_results("Histogram Bayes", hist_accuracies, hist_f1_scores,
                                                hist_all_preds, hist_all_true)


--- Fold 1/5 ---


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Accuracy: 0.4762, F1-macro: 0.1839

--- Fold 2/5 ---
Accuracy: 0.4000, F1-macro: 0.0952

--- Fold 3/5 ---
Accuracy: 0.4000, F1-macro: 0.0952

--- Fold 3/5 ---
Accuracy: 0.5500, F1-macro: 0.3486

--- Fold 4/5 ---
Accuracy: 0.5500, F1-macro: 0.3486

--- Fold 4/5 ---
Accuracy: 0.4500, F1-macro: 0.1561

--- Fold 5/5 ---
Accuracy: 0.4000, F1-macro: 0.0816

RESULTADOS CV: Histogram Bayes

Accuracy por fold: ['0.4762', '0.4000', '0.5500', '0.4500', '0.4000']
Accuracy promedio: 0.4552 ± 0.0558

F1-macro por fold: ['0.1839', '0.0952', '0.3486', '0.1561', '0.0816']
F1-macro promedio: 0.1731 ± 0.0955

--- Reporte de clasificación global (todos los folds) ---
              precision    recall  f1-score   support

      mammal       0.43      1.00      0.60        41
        bird       0.00      0.00      0.00        20
     reptile       0.00      0.00      0.00         5
        fish       1.00      0.23      0.38        13
   amphibian       0.00      0.00      0.00         4
invertebrate       

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## 8. Modelo 4: Parzen Windows

**Nota sobre selección de hiperparámetros:** 
- Se evalúan todos los hiperparámetros en cada uno de los 5 folds
- Se obtiene una matriz de resultados (hiperparámetros × folds)
- Se selecciona el hiperparámetro con mejor promedio de F1-macro
- Se reportan los resultados del mejor hiperparámetro

In [99]:
print("=" * 80)
print("4. DENSIDAD NO PARAMÉTRICA - PARZEN WINDOWS")
print("=" * 80)

class ParzenBayes:
    def __init__(self, bandwidth=0.5):
        self.bandwidth = bandwidth
        self.priors = None
        self.kdes = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.kdes = {}
        for c in self.classes:
            X_c = X[y == c].values.reshape(-1, X.shape[1])
            kde = KernelDensity(kernel='gaussian', bandwidth=self.bandwidth).fit(X_c)
            self.kdes[c] = kde
        return self
    
    def predict(self, X):
        X_val = X.values.reshape(-1, X.shape[1])
        n_samples = len(X_val)
        ll = np.zeros((n_samples, len(self.classes)))
        for i, c in enumerate(self.classes):
            ll[:, i] = np.exp(self.kdes[c].score_samples(X_val))
        posteriors = ll * self.priors
        posteriors /= posteriors.sum(axis=1, keepdims=True) + 1e-10
        return np.argmax(posteriors, axis=1)

print("✓ Clase ParzenBayes definida")

4. DENSIDAD NO PARAMÉTRICA - PARZEN WINDOWS
✓ Clase ParzenBayes definida


In [100]:
params_parzen = [0.05, 0.1, 0.5, 1.0, 1.5, 2.0]

# Matriz para almacenar resultados: [hiperparámetro][fold] = (acc, f1, preds)
parzen_results = {h: {'accuracies': [], 'f1_scores': [], 'all_preds': [], 'all_true': []} 
                  for h in params_parzen}

print("Evaluando todos los hiperparámetros en cada fold...")
print("=" * 80)

# Para cada fold, evaluar todos los hiperparámetros
for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Evaluar cada hiperparámetro en este fold
    for h in params_parzen:
        # Entrenar con este h
        parzen = ParzenBayes(bandwidth=h)
        parzen.fit(pd.DataFrame(X_train_val), y_train_val)
        
        # Evaluar en test del fold
        acc, f1, cm, preds = evaluate_fold(parzen, pd.DataFrame(X_test_fold), y_test_fold, fold)
        
        # Guardar resultados para este hiperparámetro y fold
        parzen_results[h]['accuracies'].append(acc)
        parzen_results[h]['f1_scores'].append(f1)
        parzen_results[h]['all_preds'].extend(preds)
        parzen_results[h]['all_true'].extend(y_test_fold)
        
        print(f"  h={h}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print("\n" + "=" * 80)
print("RESUMEN POR HIPERPARÁMETRO:")
print("=" * 80)

# Calcular promedios y encontrar el mejor
best_h = None
best_f1_mean = -np.inf

for h in params_parzen:
    acc_mean = np.mean(parzen_results[h]['accuracies'])
    acc_std = np.std(parzen_results[h]['accuracies'])
    f1_mean = np.mean(parzen_results[h]['f1_scores'])
    f1_std = np.std(parzen_results[h]['f1_scores'])
    
    print(f"\nh={h}:")
    print(f"  Accuracy por fold: {[f'{a:.4f}' for a in parzen_results[h]['accuracies']]}")
    print(f"  Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1-macro por fold: {[f'{f:.4f}' for f in parzen_results[h]['f1_scores']]}")
    print(f"  F1-macro: {f1_mean:.4f} ± {f1_std:.4f}")
    
    if f1_mean > best_f1_mean:
        best_f1_mean = f1_mean
        best_h = h

print("\n" + "=" * 80)
print(f"✓ MEJOR HIPERPARÁMETRO: h={best_h}")
print(f"  F1-macro: {best_f1_mean:.4f} ± {np.std(parzen_results[best_h]['f1_scores']):.4f}")
print("=" * 80)

# Usar los resultados del mejor hiperparámetro para el reporte final
parzen_accuracies = parzen_results[best_h]['accuracies']
parzen_f1_scores = parzen_results[best_h]['f1_scores']
parzen_all_preds = parzen_results[best_h]['all_preds']
parzen_all_true = parzen_results[best_h]['all_true']

parzen_mean_acc, parzen_mean_f1 = print_cv_results(f"Parzen Windows (h={best_h})", 
                                                    parzen_accuracies, parzen_f1_scores,
                                                    parzen_all_preds, parzen_all_true)

Evaluando todos los hiperparámetros en cada fold...

--- Fold 1/5 ---
  h=0.05: Accuracy=1.0000, F1-macro=1.0000


  h=0.1: Accuracy=1.0000, F1-macro=1.0000
  h=0.5: Accuracy=1.0000, F1-macro=1.0000
  h=1.0: Accuracy=0.9048, F1-macro=0.7619
  h=1.5: Accuracy=0.8095, F1-macro=0.5095
  h=2.0: Accuracy=0.7143, F1-macro=0.3550

--- Fold 2/5 ---
  h=0.05: Accuracy=0.9500, F1-macro=0.7143
  h=0.1: Accuracy=0.9500, F1-macro=0.7143
  h=0.5: Accuracy=0.9500, F1-macro=0.7143


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


  h=1.0: Accuracy=0.9500, F1-macro=0.8235
  h=1.5: Accuracy=0.8000, F1-macro=0.5856
  h=2.0: Accuracy=0.7500, F1-macro=0.4314

--- Fold 3/5 ---
  h=0.05: Accuracy=0.9500, F1-macro=0.9440
  h=0.1: Accuracy=1.0000, F1-macro=1.0000
  h=0.5: Accuracy=1.0000, F1-macro=1.0000
  h=1.0: Accuracy=0.9500, F1-macro=0.8367
  h=1.5: Accuracy=0.8000, F1-macro=0.4712
  h=0.1: Accuracy=1.0000, F1-macro=1.0000
  h=0.5: Accuracy=1.0000, F1-macro=1.0000
  h=1.0: Accuracy=0.9500, F1-macro=0.8367
  h=1.5: Accuracy=0.8000, F1-macro=0.4712
  h=2.0: Accuracy=0.7500, F1-macro=0.3856

--- Fold 4/5 ---
  h=0.05: Accuracy=0.9500, F1-macro=0.8367
  h=0.1: Accuracy=0.9500, F1-macro=0.8367
  h=0.5: Accuracy=0.9500, F1-macro=0.8367
  h=1.0: Accuracy=0.9500, F1-macro=0.8367
  h=1.5: Accuracy=0.8000, F1-macro=0.5193
  h=2.0: Accuracy=0.7500, F1-macro=0.3697

--- Fold 5/5 ---
  h=0.05: Accuracy=0.9500, F1-macro=0.9440
  h=0.1: Accuracy=0.9500, F1-macro=0.8095
  h=0.5: Accuracy=1.0000, F1-macro=1.0000
  h=1.0: Accuracy=0

## 9. Modelo 5: k-NN Density Estimator

In [101]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

class KNNDensityBayes:
    """
    Clasificador Bayesiano usando estimación de densidad k-NN.
    
    La densidad se estima como:
        p(x|ω) = k / (|D_ω| * V_k(x))
    
    donde V_k(x) es el volumen de la hiper-esfera que contiene
    los k vecinos más cercanos de x en la clase ω.
    """
    def __init__(self, k=5):
        self.k = k
        self.priors = None
        self.nn_models = {}  # Un modelo de vecinos por clase
        self.classes = None
        self.class_data = {}  # Datos de entrenamiento por clase
    
    def fit(self, X, y):
        """Entrena el modelo guardando los datos de cada clase."""
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        
        # Para cada clase, guardamos sus datos y entrenamos un NearestNeighbors
        for c in self.classes:
            X_c = X[y == c].values if hasattr(X, 'values') else X[y == c]
            self.class_data[c] = X_c
            
            # Crear modelo de vecinos más cercanos para esta clase
            nn_model = NearestNeighbors(n_neighbors=min(self.k, len(X_c)), 
                                        algorithm='auto', 
                                        metric='euclidean')
            nn_model.fit(X_c)
            self.nn_models[c] = nn_model
        
        return self
    
    def _estimate_density(self, X_val, class_idx):
        """
        Estima p(x|ω) para cada muestra en X_val usando k-NN density.
        
        p(x|ω) = k / (N_ω * V_k(x))
        donde V_k(x) es el volumen de la hiper-esfera de radio r_k(x)
        (distancia al k-ésimo vecino más cercano).
        """
        nn_model = self.nn_models[class_idx]
        N_class = len(self.class_data[class_idx])
        d = X_val.shape[1]  # dimensionalidad
        
        # Obtener distancias a los k vecinos más cercanos
        distances, _ = nn_model.kneighbors(X_val)
        
        # La distancia al k-ésimo vecino más cercano define el radio
        r_k = distances[:, -1]  # última columna = k-ésimo vecino
        
        # Volumen de hiper-esfera en d dimensiones: V_d(r) = π^(d/2) / Γ(d/2 + 1) * r^d
        # Para simplificar usamos solo r^d (la constante π^(d/2)/Γ(d/2+1) se cancela en el ratio)
        V_k = r_k ** d
        
        # Evitar divisiones por cero
        V_k = np.maximum(V_k, 1e-10)
        
        # Densidad: k / (N_class * V_k)
        density = self.k / (N_class * V_k)
        
        return density
    
    def predict(self, X):
        """Predice las clases usando la regla de Bayes."""
        X_val = X.values if hasattr(X, 'values') else X
        n_samples = len(X_val)
        
        # Calcular p(x|ω) * P(ω) para cada clase
        posteriors = np.zeros((n_samples, len(self.classes)))
        
        for i, c in enumerate(self.classes):
            likelihood = self._estimate_density(X_val, c)
            posteriors[:, i] = likelihood * self.priors[c]
        
        # Normalizar (aunque no es necesario para argmax)
        posteriors /= posteriors.sum(axis=1, keepdims=True) + 1e-10
        
        return np.argmax(posteriors, axis=1)

In [102]:
params_knn_density = [1, 3, 5, 7, 9, 11]

# Matriz para almacenar resultados: [hiperparámetro][fold] = (acc, f1, preds)
knn_d_results = {k: {'accuracies': [], 'f1_scores': [], 'all_preds': [], 'all_true': []} 
                 for k in params_knn_density}

print("Evaluando todos los hiperparámetros en cada fold...")
print("=" * 80)

# Para cada fold, evaluar todos los hiperparámetros
for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Evaluar cada hiperparámetro en este fold
    for k in params_knn_density:
        # Entrenar con este k
        knn_density = KNNDensityBayes(k=k)
        knn_density.fit(pd.DataFrame(X_train_val), y_train_val)
        
        # Evaluar en test del fold
        acc, f1, cm, preds = evaluate_fold(knn_density, pd.DataFrame(X_test_fold), y_test_fold, fold)
        
        # Guardar resultados para este hiperparámetro y fold
        knn_d_results[k]['accuracies'].append(acc)
        knn_d_results[k]['f1_scores'].append(f1)
        knn_d_results[k]['all_preds'].extend(preds)
        knn_d_results[k]['all_true'].extend(y_test_fold)
        
        print(f"  k={k}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print("\n" + "=" * 80)
print("RESUMEN POR HIPERPARÁMETRO:")
print("=" * 80)

# Calcular promedios y encontrar el mejor
best_k_density = None
best_f1_mean = -np.inf

for k in params_knn_density:
    acc_mean = np.mean(knn_d_results[k]['accuracies'])
    acc_std = np.std(knn_d_results[k]['accuracies'])
    f1_mean = np.mean(knn_d_results[k]['f1_scores'])
    f1_std = np.std(knn_d_results[k]['f1_scores'])
    
    print(f"\nk={k}:")
    print(f"  Accuracy por fold: {[f'{a:.4f}' for a in knn_d_results[k]['accuracies']]}")
    print(f"  Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1-macro por fold: {[f'{f:.4f}' for f in knn_d_results[k]['f1_scores']]}")
    print(f"  F1-macro: {f1_mean:.4f} ± {f1_std:.4f}")
    
    if f1_mean > best_f1_mean:
        best_f1_mean = f1_mean
        best_k_density = k

print("\n" + "=" * 80)
print(f"✓ MEJOR HIPERPARÁMETRO: k={best_k_density}")
print(f"  F1-macro: {best_f1_mean:.4f} ± {np.std(knn_d_results[best_k_density]['f1_scores']):.4f}")
print("=" * 80)

# Usar los resultados del mejor hiperparámetro para el reporte final
knn_d_accuracies = knn_d_results[best_k_density]['accuracies']
knn_d_f1_scores = knn_d_results[best_k_density]['f1_scores']
knn_d_all_preds = knn_d_results[best_k_density]['all_preds']
knn_d_all_true = knn_d_results[best_k_density]['all_true']

knn_d_mean_acc, knn_d_mean_f1 = print_cv_results(f"k-NN Density Bayes (k={best_k_density})", 
                                                  knn_d_accuracies, knn_d_f1_scores,
                                                  knn_d_all_preds, knn_d_all_true)

Evaluando todos los hiperparámetros en cada fold...

--- Fold 1/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000
  k=3: Accuracy=0.9524, F1-macro=0.8286
  k=5: Accuracy=0.8571, F1-macro=0.6190
  k=7: Accuracy=0.8571, F1-macro=0.6190
  k=9: Accuracy=0.8571, F1-macro=0.6190
  k=11: Accuracy=0.8571, F1-macro=0.6190

--- Fold 2/5 ---
  k=1: Accuracy=0.9500, F1-macro=0.7143
  k=3: Accuracy=0.8000, F1-macro=0.5143
  k=5: Accuracy=0.8000, F1-macro=0.5143
  k=7: Accuracy=0.8000, F1-macro=0.5143
  k=5: Accuracy=0.8000, F1-macro=0.5143
  k=7: Accuracy=0.8000, F1-macro=0.5143


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


  k=9: Accuracy=0.8000, F1-macro=0.5143
  k=11: Accuracy=0.8000, F1-macro=0.5103

--- Fold 3/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000
  k=3: Accuracy=0.8500, F1-macro=0.6224
  k=5: Accuracy=0.8500, F1-macro=0.6224
  k=7: Accuracy=0.8500, F1-macro=0.6224
  k=9: Accuracy=0.8500, F1-macro=0.6224
  k=11: Accuracy=0.8500, F1-macro=0.6224

--- Fold 4/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000
  k=3: Accuracy=0.9000, F1-macro=0.7415
  k=5: Accuracy=0.8000, F1-macro=0.6095
  k=3: Accuracy=0.9000, F1-macro=0.7415
  k=5: Accuracy=0.8000, F1-macro=0.6095
  k=7: Accuracy=0.8000, F1-macro=0.6095
  k=9: Accuracy=0.8000, F1-macro=0.6095
  k=11: Accuracy=0.8000, F1-macro=0.6056

--- Fold 5/5 ---
  k=1: Accuracy=0.9500, F1-macro=0.8095
  k=3: Accuracy=0.8000, F1-macro=0.6202
  k=5: Accuracy=0.8000, F1-macro=0.6202
  k=7: Accuracy=0.8000, F1-macro=0.6202
  k=9: Accuracy=0.8000, F1-macro=0.6202
  k=11: Accuracy=0.8000, F1-macro=0.6202

RESUMEN POR HIPERPARÁMETRO:

k=1:
  Accuracy por fold: ['1.00

## 10. Modelo 6: k-NN Rule (Directo)

In [103]:
print("=" * 80)
print("6. K-NEAREST NEIGHBORS RULE (Directo)")
print("=" * 80)

params_knn = [1, 3, 5, 7, 9, 11]

# Matriz para almacenar resultados: [hiperparámetro][fold] = (acc, f1, preds)
knn_results = {k: {'accuracies': [], 'f1_scores': [], 'all_preds': [], 'all_true': []} 
               for k in params_knn}

print("Evaluando todos los hiperparámetros en cada fold...")
print("=" * 80)

# Para cada fold, evaluar todos los hiperparámetros
for fold, (train_val_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/5 ---")
    
    # Dividir en train+val y test
    X_train_val, X_test_fold = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test_fold = y[train_val_idx], y[test_idx]
    
    # Evaluar cada hiperparámetro en este fold
    for k in params_knn:
        # Entrenar con este k
        knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
        knn.fit(X_train_val, y_train_val)
        
        # Evaluar en test del fold
        acc, f1, cm, preds = evaluate_fold(knn, X_test_fold, y_test_fold, fold)
        
        # Guardar resultados para este hiperparámetro y fold
        knn_results[k]['accuracies'].append(acc)
        knn_results[k]['f1_scores'].append(f1)
        knn_results[k]['all_preds'].extend(preds)
        knn_results[k]['all_true'].extend(y_test_fold)
        
        print(f"  k={k}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print("\n" + "=" * 80)
print("RESUMEN POR HIPERPARÁMETRO:")
print("=" * 80)

# Calcular promedios y encontrar el mejor
best_k_knn = None
best_f1_mean = -np.inf

for k in params_knn:
    acc_mean = np.mean(knn_results[k]['accuracies'])
    acc_std = np.std(knn_results[k]['accuracies'])
    f1_mean = np.mean(knn_results[k]['f1_scores'])
    f1_std = np.std(knn_results[k]['f1_scores'])
    
    print(f"\nk={k}:")
    print(f"  Accuracy por fold: {[f'{a:.4f}' for a in knn_results[k]['accuracies']]}")
    print(f"  Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1-macro por fold: {[f'{f:.4f}' for f in knn_results[k]['f1_scores']]}")
    print(f"  F1-macro: {f1_mean:.4f} ± {f1_std:.4f}")
    
    if f1_mean > best_f1_mean:
        best_f1_mean = f1_mean
        best_k_knn = k

print("\n" + "=" * 80)
print(f"✓ MEJOR HIPERPARÁMETRO: k={best_k_knn}")
print(f"  F1-macro: {best_f1_mean:.4f} ± {np.std(knn_results[best_k_knn]['f1_scores']):.4f}")
print("=" * 80)

# Usar los resultados del mejor hiperparámetro para el reporte final
knn_accuracies = knn_results[best_k_knn]['accuracies']
knn_f1_scores = knn_results[best_k_knn]['f1_scores']
knn_all_preds = knn_results[best_k_knn]['all_preds']
knn_all_true = knn_results[best_k_knn]['all_true']

knn_mean_acc, knn_mean_f1 = print_cv_results(f"k-NN Rule (k={best_k_knn})", 
                                              knn_accuracies, knn_f1_scores,
                                              knn_all_preds, knn_all_true)

6. K-NEAREST NEIGHBORS RULE (Directo)
Evaluando todos los hiperparámetros en cada fold...

--- Fold 1/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000


  k=3: Accuracy=1.0000, F1-macro=1.0000
  k=5: Accuracy=0.9524, F1-macro=0.8286
  k=7: Accuracy=0.9048, F1-macro=0.7619
  k=9: Accuracy=0.8095, F1-macro=0.5163
  k=11: Accuracy=0.8095, F1-macro=0.4877

--- Fold 2/5 ---
  k=1: Accuracy=0.9500, F1-macro=0.7143
  k=3: Accuracy=0.9500, F1-macro=0.7143
  k=5: Accuracy=0.8000, F1-macro=0.5415
  k=7: Accuracy=0.8000, F1-macro=0.5143
  k=9: Accuracy=0.8000, F1-macro=0.5903
  k=11: Accuracy=0.8000, F1-macro=0.5903

--- Fold 3/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000
  k=3: Accuracy=0.9500, F1-macro=0.8367
  k=5: Accuracy=0.9000, F1-macro=0.7415
  k=7: Accuracy=0.8500, F1-macro=0.6224
  k=9: Accuracy=0.8500, F1-macro=0.6224


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


  k=11: Accuracy=0.8000, F1-macro=0.4712

--- Fold 4/5 ---
  k=1: Accuracy=0.9500, F1-macro=0.8367
  k=3: Accuracy=0.9500, F1-macro=0.8367
  k=5: Accuracy=0.8500, F1-macro=0.7167
  k=7: Accuracy=0.8500, F1-macro=0.7167
  k=9: Accuracy=0.7500, F1-macro=0.5060
  k=11: Accuracy=0.7500, F1-macro=0.5060

--- Fold 5/5 ---
  k=1: Accuracy=1.0000, F1-macro=1.0000
  k=3: Accuracy=0.9000, F1-macro=0.7725
  k=5: Accuracy=0.8000, F1-macro=0.6202
  k=7: Accuracy=0.7500, F1-macro=0.4700
  k=9: Accuracy=0.7500, F1-macro=0.4700
  k=11: Accuracy=0.7500, F1-macro=0.4700

RESUMEN POR HIPERPARÁMETRO:

k=1:
  Accuracy por fold: ['1.0000', '0.9500', '1.0000', '0.9500', '1.0000']
  Accuracy: 0.9800 ± 0.0245
  F1-macro por fold: ['1.0000', '0.7143', '1.0000', '0.8367', '1.0000']
  F1-macro: 0.9102 ± 0.1166

k=3:
  Accuracy por fold: ['1.0000', '0.9500', '0.9500', '0.9500', '0.9000']
  Accuracy: 0.9500 ± 0.0316
  F1-macro por fold: ['1.0000', '0.7143', '0.8367', '0.8367', '0.7725']
  F1-macro: 0.8321 ± 0.0956


## 11. Comparación final de todos los modelos

Resultados de validación cruzada 5-fold sobre todo el conjunto de datos. Cada modelo se evaluó en 5 tests diferentes (uno por fold), y se reportan las métricas promedio.

In [104]:
print("=" * 80)
print("COMPARACIÓN FINAL DE MODELOS - CROSS VALIDATION 5-FOLD")
print("=" * 80)

# Compilar todos los resultados
results = {
    'Naive Bayes': {
        'acc_mean': np.mean(nb_accuracies),
        'acc_std': np.std(nb_accuracies),
        'f1_mean': np.mean(nb_f1_scores),
        'f1_std': np.std(nb_f1_scores)
    },
    'MLE Full': {
        'acc_mean': np.mean(mle_accuracies),
        'acc_std': np.std(mle_accuracies),
        'f1_mean': np.mean(mle_f1_scores),
        'f1_std': np.std(mle_f1_scores)
    },
    'Histogram Bayes': {
        'acc_mean': np.mean(hist_accuracies),
        'acc_std': np.std(hist_accuracies),
        'f1_mean': np.mean(hist_f1_scores),
        'f1_std': np.std(hist_f1_scores)
    },
    'Parzen Windows': {
        'acc_mean': np.mean(parzen_accuracies),
        'acc_std': np.std(parzen_accuracies),
        'f1_mean': np.mean(parzen_f1_scores),
        'f1_std': np.std(parzen_f1_scores)
    },
    'k-NN Density Bayes': {
        'acc_mean': np.mean(knn_d_accuracies),
        'acc_std': np.std(knn_d_accuracies),
        'f1_mean': np.mean(knn_d_f1_scores),
        'f1_std': np.std(knn_d_f1_scores)
    },
    'k-NN Rule': {
        'acc_mean': np.mean(knn_accuracies),
        'acc_std': np.std(knn_accuracies),
        'f1_mean': np.mean(knn_f1_scores),
        'f1_std': np.std(knn_f1_scores)
    }
}

print(f"\n{'Modelo':<25} {'Accuracy (mean±std)':>25} {'F1-macro (mean±std)':>25}")
print("-" * 80)
for model, metrics in results.items():
    acc_str = f"{metrics['acc_mean']:.4f} ± {metrics['acc_std']:.4f}"
    f1_str = f"{metrics['f1_mean']:.4f} ± {metrics['f1_std']:.4f}"
    print(f"{model:<25} {acc_str:>25} {f1_str:>25}")
print("-" * 80)

# Mejor modelo por F1-macro (prioridad para multiclass)
best_model = max(results, key=lambda k: results[k]['f1_mean'])
print(f"\n✓ Mejor modelo (por F1-macro promedio): {best_model}")
print(f"  F1-macro: {results[best_model]['f1_mean']:.4f} ± {results[best_model]['f1_std']:.4f}")

print("\n" + "=" * 80)
print("✓ Análisis completo finalizado con Cross-Validation 5-fold")
print("=" * 80)

COMPARACIÓN FINAL DE MODELOS - CROSS VALIDATION 5-FOLD

Modelo                          Accuracy (mean±std)       F1-macro (mean±std)
--------------------------------------------------------------------------------
Naive Bayes                         0.9600 ± 0.0583           0.8990 ± 0.1360
MLE Full                            0.8114 ± 0.0390           0.5840 ± 0.0540
Histogram Bayes                     0.4552 ± 0.0558           0.1731 ± 0.0955
Parzen Windows                      0.9800 ± 0.0245           0.9102 ± 0.1166
k-NN Density Bayes                  0.9800 ± 0.0245           0.9048 ± 0.1205
k-NN Rule                           0.9800 ± 0.0245           0.9102 ± 0.1166
--------------------------------------------------------------------------------

✓ Mejor modelo (por F1-macro promedio): Parzen Windows
  F1-macro: 0.9102 ± 0.1166

✓ Análisis completo finalizado con Cross-Validation 5-fold
